In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [11]:
cities = ['Chicago', 'Denver', 'Des Moines']

In [2]:
data = pd.read_csv('./data/processed/main.csv')
data_norm = pd.read_csv('./data/processed/main_norm.csv')

In [3]:
data.head()

,Zipcode,public_transport,shop,schools,bike_rental,parking,population_density,city
0,50309,196,100,0,24,2,1914.0,Des Moines
1,50310,6,86,0,0,0,3752.0,Des Moines
2,50311,46,34,0,8,0,6011.0,Des Moines
3,50312,52,44,0,2,2,2701.0,Des Moines
4,50313,46,10,0,0,0,963.0,Des Moines


In [25]:
count = data.groupby('city').sum()['bike_rental']
count

city
Chicago       1630
Denver          78
Des Moines      42
Name: bike_rental, dtype: int64

In [26]:
relative_score_sum = data_norm.groupby('city').sum()['relative_score']
data_norm['relative_score_norm'] = data_norm.apply(lambda x: x['relative_score'] / relative_score_sum[x['city']], axis=1)

In [27]:
data_norm.groupby('city').sum()['relative_score_norm']

city
Chicago       1.0
Denver        1.0
Des Moines    1.0
Name: relative_score_norm, dtype: float64

In [29]:
count*1.5 - count

city
Chicago       815.0
Denver         39.0
Des Moines     21.0
Name: bike_rental, dtype: float64

In [42]:
data_norm.fillna(0, inplace=True)

In [43]:
YEARS = 5
RATE = 1.3
year_plan = dict()

for year in range(YEARS):
    dcity_stations= count*(RATE**(year+1)) - count*(RATE**year)
    dzipcode_stations = data_norm.apply(lambda x: round(x['relative_score_norm'] * dcity_stations[x['city']]), axis=1)
    year_plan[year] = pd.DataFrame(
        {
            'Zipcode':data_norm['Zipcode'], 
            'city':data_norm['city'], 
            'new_stations':dzipcode_stations,
            'relative_score' : data_norm['relative_score']
        }
    )

In [45]:
year_plan[1].sample(5)

,Zipcode,city,new_stations,relative_score
39,60623,Chicago,11,0.661072
85,80204,Denver,1,0.810872
86,80205,Denver,1,0.764355
59,60645,Chicago,11,0.634827
79,80012,Denver,1,0.532404


In [50]:
for y in range(5):
    year_plan[y][['city', 'Zipcode', 'new_stations']].to_csv(f'./data/plan/year_{y+1}.csv', index=False)